In [1]:
install.packages("glmmTMB")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘nloptr’, ‘statmod’, ‘TMB’, ‘lme4’, ‘RcppEigen’




In [2]:
library(glmmTMB)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [15]:
bert_df <- read.csv("cached_df_alt.csv")
bert_df <- tidyr::separate(bert_df, SubjAttractor, c("subj", "attr"), sep=1:2)

bert_df$match <- bert_df$subj == bert_df$attr
bert_df$subj <- bert_df$subj

bert_df$verb_lang <- substr(bert_df$Langs, 3, 3)
bert_df$verb_type <- bert_df$SentIds

bert_df$match_lang <- substr(bert_df$Langs, 1, 1) == substr(bert_df$Langs, 2, 2)
bert_df$sv_match_lang <- substr(bert_df$Langs, 1, 1) == substr(bert_df$Langs, 3, 3)
bert_df$av_match_lang <- substr(bert_df$Langs, 2, 2) == substr(bert_df$Langs, 3, 3)

bert_df$monolingual <- (bert_df$verb_lang == substr(bert_df$Langs, 2, 2)) & bert_df$match_lang
# means <- bert_df %>% group_by(verb_type, verb_lang, subj, match) %>% summarize(accuracy = mean(Accuracies))

# subset(bert_df, monolingual==FALSE)

# make into factors
bert_df$Langs = as.factor(bert_df$Langs)

bert_df$verb_lang = as.factor(bert_df$verb_lang)
bert_df$match_lang = as.factor(bert_df$match_lang)
bert_df$verb_type = as.factor(bert_df$verb_type)
bert_df$subj = as.factor(bert_df$subj)
bert_df$match = as.factor(bert_df$match)
bert_df$sv_match_lang = as.factor(bert_df$sv_match_lang)
bert_df$av_match_lang = as.factor(bert_df$av_match_lang)


# # make stuff -1 or +1 so 0s don't dissapear things
# bert_df$match <- 2*(as.numeric(bert_df$match) - 1.5)
# bert_df$subj <- 2*(as.numeric(bert_df$subj) - 1.5)

# subset(bert_df, match != 1)

logit2prob <- function(logit){
  odds <- exp(logit)
  prob <- odds / (1 + odds)
  return(prob)
}

In [28]:
# ## confirm monolingual agreement attraction
# monolingual <- subset(bert_df, monolingual=="TRUE")

# # contrasts(monolingual$match) <- contr.sum(2)

# # Accuracies is binary
# bert_lm <- glmmTMB(data=monolingual, 
#                  Accuracies ~ match + (1 | SentIds) + (1 | verb_lang),
#                  family=binomial())

# summary(monolingual$match)
# summary(bert_lm)

# ## confirm monolingual agreement attraction
monolingual <- subset(bert_df, monolingual=="TRUE")
russian <- subset(monolingual, verb_lang=="R")

match_s <- subset(russian, match=="TRUE" & subj=="S")$Accuracies * 100
nomatch_s <- subset(russian, match=="FALSE" & subj=="S")$Accuracies * 100

t.test(match_s, nomatch_s)

match_s <- subset(russian, match=="TRUE" & subj=="P")$Accuracies
nomatch_s <- subset(russian, match=="FALSE" & subj=="P")$Accuracies
"SUBJ P"
t.test(match_s, nomatch_s)


# contrasts(monolingual$match) <- contr.sum(2)

# # Accuracies is binary
# bert_lm <- glmmTMB(data=subset(monolingual, verb_lang=="R"), 
#                  Accuracies ~ match + (1 | SentIds),
#                  family=binomial())

# "RUSSIAN"
# summary(monolingual$match)
# summary(bert_lm)


	Welch Two Sample t-test

data:  match_s and nomatch_s
t = 5.1077, df = 5722.4, p-value = 3.366e-07
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 3.573066 8.024156
sample estimates:
mean of x mean of y 
 78.12500  72.32639 


[1] "SUBJ P"


	Welch Two Sample t-test

data:  match_s and nomatch_s
t = 2.3831, df = 5752.4, p-value = 0.0172
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.004989056 0.051260944
sample estimates:
mean of x mean of y 
0.7361111 0.7079861 


In [59]:
# TESTING FOR THE MAIN EFFECT OF MATCH (with subject at not plural or sing)
#### Definitely an effect. thank god
# TESTING FOR THE EFFECT OF MATCH:SUBJ INTERACTION (with subject at not plural or sing)
### also def an effect

test <- bert_df
test$subj <- 2*(as.numeric(test$subj) - 1.5)

# Accuracies is binary
inner <- glmmTMB(data=test, 
                 Accuracies ~ match + subj + (1 | SentIds),
                 family=binomial())
summary(inner)
outer <- glmmTMB(data=test, 
                 Accuracies ~ match*subj + (1 | SentIds),
                 family=binomial())
summary(outer)
              
anova(inner,outer)

 Family: binomial  ( logit )
Formula:          Accuracies ~ match:subj + subj + (1 | SentIds)
Data: test

      AIC       BIC    logLik  deviance  df.resid 
 401566.6  401609.2 -200779.3  401558.6    311036 

Random effects:

Conditional model:
 Groups  Name        Variance Std.Dev.
 SentIds (Intercept) 0.1157   0.3401  
Number of obs: 311040, groups:  SentIds, 2880

Conditional model:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)     0.602997   0.007401   81.48  < 2e-16 ***
subj            0.090628   0.005360   16.91  < 2e-16 ***
subj:matchTRUE -0.048156   0.007577   -6.36 2.07e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

 Family: binomial  ( logit )
Formula:          Accuracies ~ match * subj + (1 | SentIds)
Data: test

      AIC       BIC    logLik  deviance  df.resid 
 393737.3  393790.5 -196863.6  393727.3    311035 

Random effects:

Conditional model:
 Groups  Name        Variance Std.Dev.
 SentIds (Intercept) 0.1229   0.3506  
Number of obs: 311040, groups:  SentIds, 2880

Conditional model:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)     0.280976   0.008353   33.64  < 2e-16 ***
matchTRUE       0.676788   0.007719   87.67  < 2e-16 ***
subj            0.085154   0.005195   16.39  < 2e-16 ***
matchTRUE:subj -0.037148   0.007709   -4.82 1.44e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
inner,4,401566.6,401609.2,-200779.3,401558.6,NA,NA,NA
outer,5,393737.3,393790.5,-196863.6,393727.3,7831.294,1,0


In [16]:
# TESTING FOR THE MAIN EFFECT OF av_match_lang

#english to make faster
test <- subset(bert_df)#, verb_lang=="R")

test$sv_match_lang <- 2*(as.numeric(test$sv_match_lang) - 1.5)
test$av_match_lang <- -2*(as.numeric(test$av_match_lang) - 1.5)
test$match <- 2*(as.numeric(test$match) - 1.5)

# Accuracies is binary
"SV ALONE"
inner <- glmmTMB(data=test, 
                 Accuracies ~ match*av_match_lang + (1 | SentIds) + (1 | verb_lang),
                 family=binomial())
summary(inner)
outer <- glmmTMB(data=test, 
                 Accuracies ~ match*av_match_lang*sv_match_lang + (1 | SentIds) + (1 | verb_lang),
                 family=binomial())
summary(outer)
              
anova(inner,outer)

# "SVINTERACTION"
# # Accuracies is binary
# inner <- glmmTMB(data=test, 
#                  Accuracies ~ match + av_match_lang + (1 | SentIds) + (1 | verb_lang),
#                  family=binomial())
# summary(inner)
# outer <- glmmTMB(data=test, 
#                  Accuracies ~ match*av_match_lang + (1 | SentIds) + (1 | verb_lang),
#                  family=binomial())
# summary(outer)
              
# anova(inner,outer)

[1] "SV ALONE"

 Family: binomial  ( logit )
Formula:          
Accuracies ~ match * av_match_lang + (1 | SentIds) + (1 | verb_lang)
Data: test

      AIC       BIC    logLik  deviance  df.resid 
 390795.5  390859.4 -195391.7  390783.5    311034 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 SentIds   (Intercept) 0.12564  0.3545  
 verb_lang (Intercept) 0.03727  0.1930  
Number of obs: 311040, groups:  SentIds, 2880; verb_lang, 3

Conditional model:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)          0.654720   0.111731    5.86 4.64e-09 ***
match                0.376834   0.004180   90.16  < 2e-16 ***
av_match_lang       -0.081470   0.004172  -19.53  < 2e-16 ***
match:av_match_lang -0.098072   0.004172  -23.50  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

 Family: binomial  ( logit )
Formula:          
Accuracies ~ match * av_match_lang * sv_match_lang + (1 | SentIds) +  
    (1 | verb_lang)
Data: test

      AIC       BIC    logLik  deviance  df.resid 
 388276.1  388382.6 -194128.0  388256.1    311030 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 SentIds   (Intercept) 0.12803  0.3578  
 verb_lang (Intercept) 0.03787  0.1946  
Number of obs: 311040, groups:  SentIds, 2880; verb_lang, 3

Conditional model:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                        0.714978   0.112650    6.35  2.2e-10 ***
match                              0.383439   0.004509   85.04  < 2e-16 ***
av_match_lang                     -0.051499   0.004502  -11.44  < 2e-16 ***
sv_match_lang                      0.165254   0.004503   36.70  < 2e-16 ***
match:av_match_lang               -0.079365   0.004502  -17.63  < 2e-16 ***
match:sv_match_lang                0.009716   0.004

,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
inner,6,390795.5,390859.4,-195391.7,390783.5,NA,NA,NA
outer,10,388276.1,388382.6,-194128.0,388256.1,2527.39,4,0


In [52]:
logit2prob(0.43891+0.80064)
contr.sum(2)[2,]

[1] 0.7754857

2 
-1

In [143]:
monolingual <- subset(bert_df, monolingual=="TRUE")

contrasts(monolingual$subj) <- contr.sum(2)
contrasts(monolingual$match) <- contr.sum(2)

# Accuracies is binary
# Should be (subj * match | verb_type), but time.
bert_lm <- glmmTMB(data=monolingual, 
                 Accuracies ~ subj * match + (1 | verb_type) + (1 | verb_lang),
                 family=binomial())

summary(monolingual$subj)
summary(monolingual$match)
summary(bert_lm)


# b92_lm <- glmmTMB(data=subset(b92_df, type=="lstm" & clause=="rc"), sample/100 ~ match + (1 | item) + (1 | model), family=beta_family())
# summary(b92_lm)

# lstm_rc_eff <- -summary(b92_lm)$coeff$cond[2,1]
# lstm_rc_se <- summary(b92_lm)$coeff$cond[2,2]

# b92_lm <- glmmTMB(data=subset(b92_df, type=="lstm" & clause=="prop"), sample/100 ~ match + (1 | item) + (1 | model), family=beta_family())
# summary(b92_lm)

# lstm_pp_eff <- -summary(b92_lm)$coeff$cond[2,1]
# lstm_pp_se <- summary(b92_lm)$coeff$cond[2,2]

# b92_effs <- data.frame(clause=c("RC","PP"), 
#                        eff=c(lstm_rc_eff, lstm_pp_eff),
#                        se=c(lstm_rc_se, lstm_pp_se))
# b92_effs

P     S 
17280 17280

FALSE  TRUE 
17280 17280

 Family: binomial  ( logit )
Formula:          Accuracies ~ subj * match + (1 | verb_type) + (1 | verb_lang)
Data: monolingual

     AIC      BIC   logLik deviance df.resid 
 38927.8  38978.5 -19457.9  38915.8    34554 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.44511  0.6672  
 verb_lang (Intercept) 0.08872  0.2979  
Number of obs: 34560, groups:  verb_type, 7; verb_lang, 3

Conditional model:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   0.81457    0.30552    2.67  0.00767 ** 
subj1         0.29351    0.01253   23.43  < 2e-16 ***
match1       -0.44541    0.01255  -35.49  < 2e-16 ***
subj1:match1  0.01620    0.01251    1.29  0.19539    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [144]:
# monolingual <- subset(bert_df, monolingual=="TRUE")
# monolingual$subj <- relevel(monolingual$subj, "S")

# contrasts(monolingual$subj) <- contr.sum(2)
# contrasts(monolingual$match) <- contr.sum(2)

# summary(monolingual$subj)
# summary(monolingual$match)

# monolingual$subj <- as.numeric(monolingual$subj) - 1.5

# monolingual$subj


In [9]:
monolingual <- subset(bert_df, monolingual=="TRUE")
monolingual$subj <- relevel(monolingual$subj, "P")
monolingual$match <- relevel(monolingual$match, "TRUE")

contrasts(monolingual$subj) <- contr.sum(2)
contrasts(monolingual$match) <- contr.sum(2)

summary(monolingual$subj)
summary(monolingual$match)

# mean(monolingual$subj)

"ENGLISH"
# Accuracies is binary
# Should be (subj * match | verb_type), but time.
bert_lm <- glmmTMB(data=subset(monolingual, verb_lang=="E"), 
                 Accuracies ~ subj * match + (1 | SentIds),
                 family=binomial())
summary(bert_lm)

"FRENCH"
# Accuracies is binary
# Should be (subj * match | verb_type), but time.
bert_lm <- glmmTMB(data=subset(monolingual, verb_lang=="F"), 
                 Accuracies ~ subj * match + (1 | SentIds),
                 family=binomial())
summary(bert_lm)

"RUSSIAN"
# Accuracies is binary
# Should be (subj * match | verb_type), but time.
bert_lm <- glmmTMB(data=subset(monolingual, verb_lang=="R"), 
                 Accuracies ~ subj * match + (1 | SentIds),
                 family=binomial())
summary(bert_lm)

P     S 
17280 17280

TRUE FALSE 
17280 17280

[1] "ENGLISH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ subj * match + (1 | SentIds)
Data: subset(monolingual, verb_lang == "E")

     AIC      BIC   logLik deviance df.resid 
  9975.9  10012.6  -4982.9   9965.9    11515 

Random effects:

Conditional model:
 Groups  Name        Variance  Std.Dev. 
 SentIds (Intercept) 6.259e-08 0.0002502
Number of obs: 11520, groups:  SentIds, 2880

Conditional model:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)    1.1743     0.0416   28.23  < 2e-16 ***
subj1          1.7145     0.0416   41.21  < 2e-16 ***
match1         0.9756     0.0416   23.45  < 2e-16 ***
subj1:match1   0.2347     0.0416    5.64  1.7e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "FRENCH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ subj * match + (1 | SentIds)
Data: subset(monolingual, verb_lang == "F")

     AIC      BIC   logLik deviance df.resid 
 13351.3  13388.0  -6670.6  13341.3    11515 

Random effects:

Conditional model:
 Groups  Name        Variance  Std.Dev. 
 SentIds (Intercept) 7.536e-09 8.681e-05
Number of obs: 11520, groups:  SentIds, 2880

Conditional model:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   0.82793    0.02175   38.07   <2e-16 ***
subj1        -0.38853    0.02175  -17.87   <2e-16 ***
match1        0.58933    0.02175   27.10   <2e-16 ***
subj1:match1  0.02412    0.02175    1.11    0.267    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "RUSSIAN"

 Family: binomial  ( logit )
Formula:          Accuracies ~ subj * match + (1 | SentIds)
Data: subset(monolingual, verb_lang == "R")

     AIC      BIC   logLik deviance df.resid 
 13210.2  13247.0  -6600.1  13200.2    11515 

Random effects:

Conditional model:
 Groups  Name        Variance Std.Dev.
 SentIds (Intercept) 0.2097   0.458   
Number of obs: 11520, groups:  SentIds, 2880

Conditional model:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   1.08652    0.02582   42.08  < 2e-16 ***
subj1        -0.08366    0.02169   -3.86 0.000115 ***
match1        0.11756    0.02170    5.42 6.06e-08 ***
subj1:match1 -0.04452    0.02168   -2.05 0.040038 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [141]:
logit2prob(4.4-2.5-4.2+.7)

[1] 0.1679816

In [146]:
monolingual <- bert_df

monolingual$subj <- relevel(monolingual$subj, "P")
monolingual$match <- relevel(monolingual$match, "TRUE")

contrasts(monolingual$subj) <- contr.sum(2)
contrasts(monolingual$match) <- contr.sum(2)
contrasts(monolingual$verb_lang) <- contr.sum(3)

summary(monolingual$subj)
summary(monolingual$match)
summary(monolingual$verb_lang)

"ALL"
# Accuracies is binary
# Should be (subj * match | verb_type), but time.
bert_lm <- glmmTMB(data=monolingual, 
                 Accuracies ~ subj * match * verb_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)


P      S 
155520 155520

TRUE  FALSE 
155520 155520

E      F      R 
103680 103680 103680

[1] "ALL"

 Family: binomial  ( logit )
Formula:          Accuracies ~ subj * match * verb_lang + (1 | verb_type)
Data: monolingual

      AIC       BIC    logLik  deviance  df.resid 
 356103.9  356242.3 -178038.9  356077.9    311027 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.1322   0.3636  
Number of obs: 311040, groups:  verb_type, 7

Conditional model:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              0.601542   0.137517    4.37 1.22e-05 ***
subj1                   -0.062394   0.004199  -14.86  < 2e-16 ***
match1                   0.385279   0.004201   91.70  < 2e-16 ***
verb_lang1              -0.009687   0.006116   -1.58    0.113    
verb_lang2              -0.252455   0.006359  -39.70  < 2e-16 ***
subj1:match1             0.030944   0.004196    7.37 1.65e-13 ***
subj1:verb_lang1         0.931472   0.006043  154.14  < 2e-16 ***
subj1:verb_lang2        -0.824919   0.005947 -138.71  < 2e-16 **

In [154]:
bert_df$sv_match_lang <- relevel(bert_df$sv_match_lang, "TRUE")
bert_df$match <- relevel(bert_df$match, "TRUE")

contrasts(bert_df$sv_match_lang) <- contr.sum(2)
contrasts(bert_df$match) <- contr.sum(2)

summary(bert_df$sv_match_lang)
summary(bert_df$match)


"ENGLISH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="E"), 
                 Accuracies ~ match * sv_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"FRENCH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="F"), 
                 Accuracies ~ match * sv_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"RUSSIAN"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="R"), 
                 Accuracies ~ match * sv_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)


TRUE  FALSE 
103680 207360

TRUE  FALSE 
155520 155520

[1] "ENGLISH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match * sv_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "E")

     AIC      BIC   logLik deviance df.resid 
129186.7 129234.5 -64588.4 129176.7   103675 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.2577   0.5076  
Number of obs: 103680, groups:  verb_type, 7

Conditional model:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)           0.565877   0.192013    2.95  0.00321 ** 
match1                0.386813   0.007226   53.53  < 2e-16 ***
sv_match_lang1        0.132986   0.007210   18.44  < 2e-16 ***
match1:sv_match_lang1 0.045671   0.007209    6.34 2.36e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "FRENCH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match * sv_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "F")

     AIC      BIC   logLik deviance df.resid 
134757.5 134805.3 -67373.8 134747.5   103675 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.01785  0.1336  
Number of obs: 103680, groups:  verb_type, 5

Conditional model:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)           0.528143   0.060219    8.77   <2e-16 ***
match1                0.312220   0.007174   43.52   <2e-16 ***
sv_match_lang1        0.324652   0.007174   45.25   <2e-16 ***
match1:sv_match_lang1 0.072948   0.007173   10.17   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "RUSSIAN"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match * sv_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "R")

     AIC      BIC   logLik deviance df.resid 
117813.9 117861.7 -58902.0 117803.9   103675 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.2975   0.5454  
Number of obs: 103680, groups:  verb_type, 7

Conditional model:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)            0.900703   0.206312    4.37 1.27e-05 ***
match1                 0.381101   0.007692   49.55  < 2e-16 ***
sv_match_lang1         0.108571   0.007675   14.15  < 2e-16 ***
match1:sv_match_lang1 -0.050762   0.007674   -6.62 3.71e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [155]:
bert_df$sv_match_lang <- relevel(bert_df$sv_match_lang, "TRUE")
bert_df$match <- relevel(bert_df$match, "FALSE")

contrasts(bert_df$sv_match_lang) <- contr.sum(2)
contrasts(bert_df$match) <- contr.sum(2)

summary(bert_df$sv_match_lang)
summary(bert_df$match)


"ENGLISH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="E"), 
                 Accuracies ~ match * sv_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"FRENCH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="F"), 
                 Accuracies ~ match * sv_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"RUSSIAN"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="R"), 
                 Accuracies ~ match * sv_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)


TRUE  FALSE 
103680 207360

FALSE   TRUE 
155520 155520

[1] "ENGLISH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match * sv_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "E")

     AIC      BIC   logLik deviance df.resid 
129186.7 129234.5 -64588.4 129176.7   103675 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.2577   0.5076  
Number of obs: 103680, groups:  verb_type, 7

Conditional model:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)            0.565877   0.192013    2.95  0.00321 ** 
match1                -0.386813   0.007226  -53.53  < 2e-16 ***
sv_match_lang1         0.132986   0.007210   18.44  < 2e-16 ***
match1:sv_match_lang1 -0.045671   0.007209   -6.34 2.36e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "FRENCH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match * sv_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "F")

     AIC      BIC   logLik deviance df.resid 
134757.5 134805.3 -67373.8 134747.5   103675 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.01785  0.1336  
Number of obs: 103680, groups:  verb_type, 5

Conditional model:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)            0.528143   0.060219    8.77   <2e-16 ***
match1                -0.312220   0.007174  -43.52   <2e-16 ***
sv_match_lang1         0.324652   0.007174   45.25   <2e-16 ***
match1:sv_match_lang1 -0.072948   0.007173  -10.17   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "RUSSIAN"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match * sv_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "R")

     AIC      BIC   logLik deviance df.resid 
117813.9 117861.7 -58902.0 117803.9   103675 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.2975   0.5454  
Number of obs: 103680, groups:  verb_type, 7

Conditional model:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)            0.900703   0.206312    4.37 1.27e-05 ***
match1                -0.381101   0.007692  -49.55  < 2e-16 ***
sv_match_lang1         0.108571   0.007675   14.15  < 2e-16 ***
match1:sv_match_lang1  0.050762   0.007674    6.62 3.71e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [159]:
## IS THERE AGREEMENT ATTRACTION WHEN ITS LxL??
lxl_df <- subset(bert_df, sv_match_lang=="TRUE")
lxl_df$match <- relevel(lxl_df$match, "TRUE")

contrasts(lxl_df$match) <- contr.sum(2)
summary(lxl_df$match)


"ENGLISH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(lxl_df, verb_lang=="E"), 
                 Accuracies ~ match + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"FRENCH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(lxl_df, verb_lang=="F"), 
                 Accuracies ~ match + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"RUSSIAN"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(lxl_df, verb_lang=="R"), 
                 Accuracies ~ match + (1 | verb_type),
                 family=binomial())
summary(bert_lm)


TRUE FALSE 
51840 51840

[1] "ENGLISH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match + (1 | verb_type)
Data: subset(lxl_df, verb_lang == "E")

     AIC      BIC   logLik deviance df.resid 
 40202.7  40228.1 -20098.4  40196.7    34557 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.51     0.7141  
Number of obs: 34560, groups:  verb_type, 7

Conditional model:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.72388    0.27023    2.68  0.00739 ** 
match1       0.44954    0.01223   36.75  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "FRENCH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match + (1 | verb_type)
Data: subset(lxl_df, verb_lang == "F")

     AIC      BIC   logLik deviance df.resid 
 39707.9  39733.3 -19851.0  39701.9    34557 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.2389   0.4887  
Number of obs: 34560, groups:  verb_type, 5

Conditional model:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.84362    0.21895    3.85 0.000117 ***
match1       0.39993    0.01233   32.43  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "RUSSIAN"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match + (1 | verb_type)
Data: subset(lxl_df, verb_lang == "R")

     AIC      BIC   logLik deviance df.resid 
 36229.7  36255.0 -18111.8  36223.7    34557 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.8276   0.9097  
Number of obs: 34560, groups:  verb_type, 7

Conditional model:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.16318    0.34418    3.38 0.000726 ***
match1       0.34647    0.01301   26.64  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [158]:
## IS THERE AGREEMENT ATTRACTION WHEN SUBJ & VERB DIFFERENT LANGS ??
lxl_df <- subset(bert_df, sv_match_lang=="FALSE")
lxl_df$match <- relevel(lxl_df$match, "TRUE")

contrasts(lxl_df$match) <- contr.sum(2)
summary(lxl_df$match)


"ENGLISH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(lxl_df, verb_lang=="E"), 
                 Accuracies ~ match + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"FRENCH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(lxl_df, verb_lang=="F"), 
                 Accuracies ~ match + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"RUSSIAN"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(lxl_df, verb_lang=="R"), 
                 Accuracies ~ match + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

TRUE  FALSE 
103680 103680

[1] "ENGLISH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match + (1 | verb_type)
Data: subset(lxl_df, verb_lang == "E")

     AIC      BIC   logLik deviance df.resid 
 88705.8  88733.3 -44349.9  88699.8    69117 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.1797   0.4239  
Number of obs: 69120, groups:  verb_type, 7

Conditional model:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) 0.431528   0.160421    2.69  0.00715 ** 
match1      0.336128   0.008047   41.77  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "FRENCH"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match + (1 | verb_type)
Data: subset(lxl_df, verb_lang == "F")

     AIC      BIC   logLik deviance df.resid 
 93873.3  93900.7 -46933.6  93867.3    69117 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.00625  0.07906 
Number of obs: 69120, groups:  verb_type, 5

Conditional model:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.22559    0.03628   6.218 5.04e-10 ***
match1       0.23864    0.00772  30.912  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "RUSSIAN"

 Family: binomial  ( logit )
Formula:          Accuracies ~ match + (1 | verb_type)
Data: subset(lxl_df, verb_lang == "R")

     AIC      BIC   logLik deviance df.resid 
 80293.6  80321.0 -40143.8  80287.6    69117 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.2331   0.4828  
Number of obs: 69120, groups:  verb_type, 7

Conditional model:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) 0.764127   0.182710    4.18 2.89e-05 ***
match1      0.428932   0.008668   49.48  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [161]:
## IS THERE AGREEMENT ATTRACTION WHEN SUBJ & VERB DIFFERENT LANGS ??
bert_df$match <- relevel(bert_df$match, "FALSE")
bert_df$sv_match_lang <- relevel(bert_df$sv_match_lang, "FALSE")
bert_df$av_match_lang <- relevel(bert_df$av_match_lang, "TRUE")

contrasts(bert_df$sv_match_lang) <- contr.sum(2)
contrasts(bert_df$av_match_lang) <- contr.sum(2)
contrasts(bert_df$match) <- contr.sum(2)
summary(bert_df$sv_match_lang)
summary(bert_df$av_match_lang)
summary(bert_df$match)

"ENGLISH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="E"), 
                 Accuracies ~ match*sv_match_lang*av_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"FRENCH"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="F"), 
                 Accuracies ~ match*sv_match_lang*av_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

"RUSSIAN"
# Accuracies is binary
bert_lm <- glmmTMB(data=subset(bert_df, verb_lang=="R"), 
                 Accuracies ~ match*sv_match_lang*av_match_lang + (1 | verb_type),
                 family=binomial())
summary(bert_lm)

FALSE   TRUE 
207360 103680

TRUE  FALSE 
103680 207360

FALSE   TRUE 
155520 155520

[1] "ENGLISH"

 Family: binomial  ( logit )
Formula:          
Accuracies ~ match * sv_match_lang * av_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "E")

     AIC      BIC   logLik deviance df.resid 
128318.6 128404.5 -64150.3 128300.6   103671 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.2623   0.5121  
Number of obs: 103680, groups:  verb_type, 7

Conditional model:
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           0.5696407  0.1937262    2.94 0.003277 ** 
match1                               -0.4472577  0.0077625  -57.62  < 2e-16 ***
sv_match_lang1                       -0.1214722  0.0077417  -15.69  < 2e-16 ***
av_match_lang1                        0.0002512  0.0077401    0.03 0.974105    
match1:sv_match_lang1                 0.0176650  0.0077400    2.28 0.022472 *  
match1:av_match_lang1                -0.1719355  0.0077434  -22.20  < 2e-16 ***
sv_mat

[1] "FRENCH"

 Family: binomial  ( logit )
Formula:          
Accuracies ~ match * sv_match_lang * av_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "F")

     AIC      BIC   logLik deviance df.resid 
133637.3 133723.3 -66809.7 133619.3   103671 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.01825  0.1351  
Number of obs: 103680, groups:  verb_type, 5

Conditional model:
                                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           0.543438   0.060944    8.92  < 2e-16 ***
match1                               -0.378491   0.007709  -49.10  < 2e-16 ***
sv_match_lang1                       -0.292692   0.007708  -37.97  < 2e-16 ***
av_match_lang1                        0.031365   0.007707    4.07 4.71e-05 ***
match1:sv_match_lang1                 0.053738   0.007707    6.97 3.12e-12 ***
match1:av_match_lang1                -0.189345   0.007708  -24.57  < 2e-16 ***
sv_match_lang

[1] "RUSSIAN"

 Family: binomial  ( logit )
Formula:          
Accuracies ~ match * sv_match_lang * av_match_lang + (1 | verb_type)
Data: subset(bert_df, verb_lang == "R")

     AIC      BIC   logLik deviance df.resid 
116655.4 116741.4 -58318.7 116637.4   103671 

Random effects:

Conditional model:
 Groups    Name        Variance Std.Dev.
 verb_type (Intercept) 0.3054   0.5526  
Number of obs: 103680, groups:  verb_type, 7

Conditional model:
                                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           0.951027   0.209049    4.55 5.38e-06 ***
match1                               -0.329575   0.008229  -40.05  < 2e-16 ***
sv_match_lang1                       -0.068803   0.008216   -8.37  < 2e-16 ***
av_match_lang1                        0.130180   0.008218   15.84  < 2e-16 ***
match1:sv_match_lang1                -0.051866   0.008216   -6.31 2.74e-10 ***
match1:av_match_lang1                 0.148955   0.008219   18.12  < 2e-16 ***
sv_match_lang